# **Text completions with Writer**

**_Text completions_** are LLM responses to a user’s request, command or question. They tend to be longer than chat completions, and you can specify a maximum response length. This cookbook shows how to use the `create()` method of Writer’s `completions` object to create apps that provide text completions, with streaming and non-streaming responses.

## **Contents**

- [Introduction](#introduction)
- [Setup](#setup)
- [The `completions` object](#the-completions-object)
- [Text completion (non-streaming version)](#text-completion-non-streaming-version)
- [Text completion (streaming version)](#text-completion-streaming-version)
- [For more information](#for-more-information)

<a id="introduction"></a>
## **Introduction**

### What are text completions?

**In text completion, the model generates or completes text based on a given prompt.** It’s usually a single-turn process, where the user provides an input and the model generates an output, and the model doesn’t have to maintain context or history over multiple inputs and outputs. It’s “one and done.”

Text completion, as its name implies, is often used for completing sentences and paragraphs, but can also be used to generate long-form text content based on a single prompt or some kind of partial input. You may want to think of text completions as longer-form versions of “one-shot” chat completions.

<a id="setup"></a>
## **Setup**

### Dependencies

This notebook uses the following packages:

* `ipywidgets`: To draw UI widgets for the streaming versions of the apps.
* `python-dotenv`: To load environment variables.
* `writer-sdk`: To access the Writer API.

Run the cell below ensure you have these packages.

In [ ]:
%pip install -r requirements.txt -q

### Initialization

The cell below performs the initialization required for this notebook including the creation of an instance of the `Writer` object to interact with the LLM.

To create a Palmyra client object, you need an API key. [You can sign up for one for free](https://app.writer.com/aistudio/signup). 

Once you have an API key, we recommend that you store it as an environment variable in a `.env` file like so:

```
WRITER_API_KEY="{Your Writer API key goes here}"
```

When you instantiate the client with `client = Writer()`, the newly-created object will automatically look for an environment variable named `WRITER_API_KEY` and will complete the instantiation if an only if `WRITER_API_KEY` has been defined. This notebook uses the [python-dotenv](https://pypi.org/project/python-dotenv/) library to automatically define environment variables based on the contents of an `.env` file in the same directory.

The `Writer()` initializer method also has an `api_key` parameter that you can use like this...

```
client = Writer(api_key="{Your Writer API key goes here}")
```

...but we strongly encourage you not to leave API keys in your source code.

In [ ]:
# Run this cell before running any other cells in this cookbook!

from writerai import Writer

# Load environment variables from .env file
%reload_ext dotenv
%dotenv

client = Writer()

<a id="the-completions-object"></a>
## **The `completions` object**

Now that you have a Writer client instance, it’s time to start building text completion apps! 

The `completions` property of a Writer client instance contains methods and properties related to text completion. In all the examples in this cookbook, you’ll build text completion apps by using the `completions` property’s `create()` method, which makes requests for text completions from Palmyra.

<a id="text-completion-non-streaming-version"></a>
## **Text completion (non-streaming version)**

The cell below contains a simple one-shot text completion app. When you run it, you will be asked to enter a prompt. After you enter the prompt, you can expect to wait a few moments while Palmyra generates the complete text of its response. Once Palmyra’s done generating, the app will display the response and finish running.

Try entering a question or command that can be answered or satisfied with a few short paragraphs (e.g. “Tell me a story”).

In [ ]:
print("""
Sample text completion app
==========================
""")

while True:
    prompt = input("Enter a prompt: ").strip()
    if prompt:
        break
    print("Please provide a prompt to continue.\n")

completion = client.completions.create(
    prompt=prompt,
    temperature=0.8,
    max_tokens=2048,
    model="palmyra-x-004",
    stream=False
)
print(f"\n{completion.choices[0].text}\n\n")

### Notes

#### Calling the `create()` method

The heart of the one-shot chat completion app is this line:

```
completion = client.completions.create(
    prompt=prompt,
    temperature=0.8,
    max_tokens=2048,
    model="palmyra-x-004",
    stream=False
)
```

It makes a call to the client instance’s `completions` object’s `create()` method, which requests a text completion from Palmyra. In order to get that completion, the call provides arguments for the following parameters:

<table width="66%">
    <tr>
        <th width="25%" style="background-color: #5551ff; color: #ffffff;">Parameter</th>
        <th style="background-color: #5551ff; color: #ffffff;">Description</th>
    </tr>
    <tr>
        <td style="border: 1px solid #bfcbff;"><code>prompt</code></td>
        <td style="border: 1px solid #bfcbff;">
            The input that Palmyra will use as a basis for the completion it will return as its response.
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid #bfcbff;"><code>temperature</code></td>
        <td style="border: 1px solid #bfcbff;">
            A float that controls the level of randomness in the text that Palmyra
            generates:
            <ul>
                <li>At lower temperatures — between 0.0 and 0.5 — the responses are more deterministic and predictable, with Palmyra tending to choose the highest probability tokens based on previously-generated ones. The generated output is predictable and repetitive, and produces more "safe" or "obvious" answers.</li>
                <li>At higher temperatures — 0.8 and above — the responses are more random and “imaginative,”  with Palmyra giving less probable tokens a better chance of being chosen. The generated output is predictable and repetitive, and produces more "safe" or "obvious" answers. The generated output is less predictable, and produces more “creative” answers. The results become increasingly nonsensical at temperatures of about 1.4 and higher.</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid #bfcbff;"><code>max_tokens</code></td>
        <td style="border: 1px solid #bfcbff;">
            <p>An integer specifying the upper limit of tokens Palmyra should use to generate
            the completion. The larger this number, the longer the output, and the longer
            it will take to generate that output.</p>
            <p>A good general guideline is that one token accounts for about three-quarters
            of a word. Shorter, more common words (e.g. "cat", "me") are usually represented
            by a single token, while longer, more complex words (e.g. "complicated", 
            "incredible") might be represented by multiple tokens.</p>
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid #bfcbff;"><code>model</code></td>
        <td style="border: 1px solid #bfcbff;">
            A string specifying which model to use. In this case, we’re using the latest model
            at the time of writing, <code>palmyra-x-004</code>.
        </td>
    </tr>
    <tr>
        <td style="border: 1px solid #bfcbff;"><code>stream</code></td>
        <td style="border: 1px solid #bfcbff;">
            A boolean specifying if the method should stream the completion in chunks
            in real time as Palmyra generates it (<code>True</code>) or wait until Palmyra
            finishes generating the completion before returning a value (<code>False</code>).
            Since we want the completion all at once, we set this parameter to <code>False</code>.
        </td>
    </tr>
</table>

<a id="text-completion-streaming-version"></a>
## **Text completion (streaming version)**

The cell below is a _streaming_ version of the text completion app. With this app, after you enter the prompt, you will immediately see the completion as Palmyra generates it as a stream of text in a manner similar to a lot of AI chat apps.

Note that this version of the app uses the [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/stable/) library to provide a graphical user interface for the app. There’s a reason, which will be explained in the notes in the cell after the code.

In [ ]:
from ipywidgets import Button, ButtonStyle, HBox, Layout, Text, Textarea, VBox

def display_ui():
    prompt_text_box = Text(
        value="",
        placeholder="Enter your prompt here",
        description="Prompt:",
        layout=Layout(width="350px"),
        continuous_update=False,
        disabled=False   
    )
    submit_button = Button(
        description="Submit",
        tooltip="Click me",
        style=ButtonStyle(button_color="thistle", font_weight="bold"),
        icon="upload",
        disabled=False
    )
    completion_text_area = Textarea(
        value="",
        placeholder="",
        description="String:",
        layout=Layout(width="800px", height="200px"),
        disabled=False
    )
    display(
        VBox(
            [
                HBox([prompt_text_box, submit_button]),
                completion_text_area,
            ]
        )
    )
    return (prompt_text_box, submit_button, completion_text_area)

def generate_completion(prompt_text_box, submit_button, completion_text_area):
    # Put UI in "generating" mode
    prompt_text_box.disabled = True
    submit_button.disabled = True
    submit_button.description = "Generating..."
    submit_button.icon = "hourglass"
    completion_text_area.value = ""

    # Generate completion and display it
    completion = client.completions.create(
      model="palmyra-x-004",
      prompt=prompt_text_box.value.strip(),
      max_tokens=2048,
      temperature=0.8,
      stream=True
    )
    output_text = ""
    for chunk in completion:
        if chunk.value is None:
            continue
        else:
            completion_text_area.value += chunk.value

    # Reset UI to "Awaiting user input" mode
    prompt_text_box.disabled = False
    submit_button.disabled = False
    submit_button.description = "Submit"
    submit_button.icon = "upload"


def main():
    prompt_text_box, submit_button, completion_text_area = display_ui()
    submit_button.on_click(lambda button: generate_completion(prompt_text_box, submit_button, completion_text_area))

main()

### Notes

#### A different argument for the `create()` method

This version of the app calls the `create()` method in pretty much the same way with one notable exception: the argument it provides for the `stream` parameter is `True`, which specifies that Palmyra should stream its responses as it generates them rather than waiting until the response has been completely generated before returning it:

```
completion = client.completions.create(
    model="palmyra-x-004",
    prompt=prompt,
    max_tokens=2048,
    temperature=0.8,
    stream=True
)
```

#### Why does this version use Jupyter Widgets?

It _is_ possible to simply use a `print()` function to display the response stream — it’s as simple as this:

```
for chunk in completion:
    print(chunk.value)
```

The `for` loop continues as long as the stream hasn’t finished, with each iteration happening when Palmyra generates the next part of its response. The problem is that if you use the `print()` function to display the chunks as they arrive, you get output that looks like this:

```
 Sure
,
 I
'
d
 be
 happy
 to
 share
 a
 story
!
 Once
 upon
 a
 time
 in
 a
 small
 town
 nest
led
 between
 rolling
 hills
 and
 a
 spark
ling
 river
,
```

The solution to this problem is to feed the stream into a UI component whose contents can be updated in real time. Fortunately, there’s Jupyter Widgets, a library that brings UI widgets to Jupyter Notebooks so that they can be as interactive as web and desktop applications.

#### Drawing the UI

The `display_ui()` function creates three UI objects:

- `prompt_text_box`: A text box where the user enters their prompt.
- `submit_button`: The user clicks this button to submit their prompt.
- `completion_text_area`: Palymra’s response appears here.

The `prompt_text_box` and `submit_button` are laid out inside an `HBox` layout container, which in turn is laid out with `completion_text_area` inside a `VBox` layout container.

`display_ui()` also returns `prompt_text_box`, `submit_button`, `completion_text_area` widgets so that they can be referenced by other code.

<a id="for-more-information"></a>
## **For more information**

For more information about chat completions, the `chat` object, and its `chat()` method, see:

- [The _Text generation_ guide](https://dev.writer.com/api-guides/text-generation)
- [The completion API’s _Text generation_ page](https://dev.writer.com/api-guides/api-reference/completion-api/text-generation)